### Import Libraries

In [2]:
import os
from textwrap import dedent
from dotenv import load_dotenv
import pandas as pd
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import markdown

# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), verbose=True, override=True)

# Load environment variables
load_dotenv()

os.environ["AZURE_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")
os.environ["AZURE_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION") 
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")


### Generate Markdown file for Balance Sheet

In [3]:
## Generate Markdown file
def extract_text_from_excel(file, sheetnames):
    if len(sheetnames)>0:
        for sheet in sheetnames:
            df = pd.read_excel(file, engine='openpyxl', sheet_name=sheet)
            df_cleaned = df.dropna(how='all')
            df2 = df_cleaned.fillna('').reset_index(drop=True)
            with open("../data/processed_inputs/balancesheet.md", "a") as f:
                f.write(f"""##### {sheet} \n{str(df2.to_markdown())}\n""")
                # f.write(str(df2.to_markdown()))
    else:
        print("SheetNames not provided")

In [4]:
file_path = '../data/1. CMA_Data.xlsx'
extract_text_from_excel(file_path,sheetnames=['Balance Sheet','Balance Sheet2'])

### Define the LangGraph Agent for BalanceSheet Analyzer

In [5]:
# Define the AgentState class for BalanceSheet Analyzer
class AgentState(TypedDict):
    question: str
    balance_sheet_data: str
    result: str

def balancesheet_analyzer_agent(state: AgentState):
    """
    Financial Analyzer Agent takes input questions and responds to the user's questions by processing the data provided.

    Returns:
    dict: The response from the LLM.
    """
    # Input User Question
    question = state["question"]

    with open("../data/processed_inputs/balancesheet.md", "r") as f:
        state['balance_sheet_data'] = f.read()

    # print(state["balace_sheet_data"])

    balancesheet_data = state['balance_sheet_data']

    if balancesheet_data is None:
        print("Failed to extract text from the image.")
        return {"error": "Failed to extract text from the image."}
    
    # system = dedent(""" 
    # You are a Financial Analyst. Analyze the balance sheet and help answer questions based on the balance sheet data.
    # Provide a clean data insight for the question based on the data.
    # """)

    # system = dedent("""
    #     You are a financial analyst reviewing a company's balance sheet data across multiple time periods.
        
    #     Please provide a comprehensive analysis of the below key metrics to Monitor in Balance Sheet:
    #     1. Overall financial health assessment over time
    #     2. Analysis of key trends in liabilities and assets
    #     3. Liquidity trend analysis
    #     4. Solvency and leverage trend analysis
    #     5. Projections analysis - how realistic are the future projections?
    #     6. Key areas of concern or opportunity
    #     7. Recommendations for improving financial position
        
    #     Focus on key financial metrics, including liquidity, solvency, profitability, and efficiency.
    # """)

    system = dedent("""
        You are a financial analyst reviewing a company's balance sheet data across multiple time periods.
        Analyze the provided balance sheet to assess the company's financial health. Calculate and interpret:
        - Liquidity: Current Ratio, Quick Ratio, Working Capital.   
        - Solvency: Debt-to-Equity Ratio, Interest Coverage Ratio.
        - Efficiency: ROA, Fixed Asset Turnover.
        - Profitability: ROE, Gross Profit Margin.
        - Capital Structure: Equity Ratio, Debt Ratio.
        - Cash & Reserves: Analyze Cash/Equivalents & Retained Earnings.
        Identify trends (if data available), risks, and provide an overall assessment of financial stability and long-term sustainability, considering industry context.
        """)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", f"Balance Sheet Data: {balancesheet_data}"),
            ("human", f"Question: {question}"),
        ]
    )

    # Initialize the LLM
    llm = AzureChatOpenAI(
        model="gpt-4o-mini",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_version=os.getenv("AZURE_API_VERSION"),
        temperature=0
    )

    balancesheet_analyst = prompt | llm | StrOutputParser()

    result = balancesheet_analyst.invoke({"question": question, "balancesheet_data": balancesheet_data})
    state["result"] = result
    return state

workflow = StateGraph(AgentState)

workflow.add_node("balancesheet_analyzer_node", balancesheet_analyzer_agent)

workflow.set_entry_point("balancesheet_analyzer_node")

app = workflow.compile()


### User Questions

In [6]:
user_question_1 = "What are the total current assets?"
result = app.invoke({"question": user_question_1})
print(result["result"])

To find the total current assets, we can refer to the balance sheet data provided. The total current assets are listed under the "Total Current Assets" row.

From the data, the total current assets for the various periods are as follows:

- 31.03.2022: 257.79 lacs
- 31.03.2023: 380.31 lacs
- 2024-03-31: 508.49 lacs
- 2025-03-31: 654.78 lacs
- 2026-03-31: 819.86 lacs
- 2027-03-31: 1003.67 lacs
- 31.03.2028: 1206.72 lacs
- 31.03.2029: 1441.88 lacs

Thus, the total current assets are increasing over the periods, indicating a positive trend in liquidity.


In [6]:
user_question_2 = "What is the value of long-term debt?"
result = app.invoke({"question": user_question_2})
print(result["result"])

To determine the value of long-term debt from the provided balance sheet data, we need to look for the relevant line items that represent long-term liabilities. 

From the balance sheet data, we can identify the following relevant entries:

1. **Term Loans**: 
   - As of 31.03.2023, the value is **660.23 lacs**.
   
2. **Debentures**: 
   - The value is **0 lacs** as of 31.03.2023.

3. **Other Term Liabilities**: 
   - The value is **90 lacs** as of 31.03.2023.

Adding these values together gives us the total long-term debt:

- Term Loans: 660.23 lacs
- Debentures: 0 lacs
- Other Term Liabilities: 90 lacs

**Total Long-Term Debt = 660.23 + 0 + 90 = 750.23 lacs**

Thus, the value of long-term debt as of 31.03.2023 is **750.23 lacs**.


In [7]:
user_question_3 = "How much short-term debt does the company have?"
result = app.invoke({"question": user_question_3})
print(result["result"])

To determine the short-term debt of the company, we need to look at the current liabilities section of the balance sheet. Based on the provided data, the relevant entries for short-term debt include:

1. **Bank Borrowings**: 
   - From C.B.I.: 50 (as of 31.03.2023)
   - From others: 0

2. **Short-term borrowings from others**: 0

3. **Sundry Creditors (Trade)**: 5.68 (as of 31.03.2023)

4. **Other Current Liabilities**: 
   - Other Statutory liabilities: 5 (as of 31.03.2023)

5. **Total Current Liabilities**: 194.97 (as of 31.03.2023)

From the data, the short-term debt primarily consists of the bank borrowings and sundry creditors. 

Calculating the total short-term debt:

- Bank Borrowings (C.B.I.): 50
- Sundry Creditors (Trade): 5.68

Total Short-term Debt = Bank Borrowings + Sundry Creditors
Total Short-term Debt = 50 + 5.68 = **55.68 lacs**

Thus, the company has a total short-term debt of **55.68 lacs** as of 31.03.2023.


In [8]:
user_question_4 = "What is the current ratio?"
result = app.invoke({"question": user_question_4})
print(result["result"])

To calculate the Current Ratio, we need the values for Current Assets and Current Liabilities from the balance sheet.

From the provided balance sheet data:

- **Total Current Assets** (as of 31.03.2023): 380.31 lacs
- **Total Current Liabilities** (as of 31.03.2023): 194.97 lacs

The Current Ratio is calculated using the formula:

\[
\text{Current Ratio} = \frac{\text{Current Assets}}{\text{Current Liabilities}}
\]

Substituting the values:

\[
\text{Current Ratio} = \frac{380.31}{194.97} \approx 1.95
\]

### Interpretation:
A Current Ratio of approximately 1.95 indicates that the company has nearly twice as many current assets as current liabilities. This suggests a strong liquidity position, meaning the company should be able to cover its short-term obligations without financial strain. Generally, a Current Ratio above 1 is considered healthy, while a ratio significantly above 1 (like 1.95) is viewed positively.


In [9]:
user_question_5 = "What are the total non-current liabilities?"
result = app.invoke({"question": user_question_5})
print(result["result"])

To determine the total non-current liabilities from the provided balance sheet data, we need to identify the relevant entries under the liabilities section that are classified as non-current.

From the balance sheet data, the non-current liabilities typically include:

1. **Term Loans**
2. **Debentures**
3. **Redeemable Preference Shares**
4. **Other Non-Current Liabilities**

Based on the data provided, we can extract the values for these categories:

- **Term Loans**: 
  - 31.03.2022: 794.52
  - 31.03.2023: 660.23
  - 2024-03-31: 525.94
  - 2025-03-31: 391.65
  - 2026-03-31: 257.36
  - 2027-03-31: 123.07
  - 31.03.2028: 0
  - 31.03.2029: 0

- **Debentures**: 
  - 31.03.2022: 0
  - 31.03.2023: 0
  - 2024-03-31: 0
  - 2025-03-31: 0
  - 2026-03-31: 0
  - 2027-03-31: 0
  - 31.03.2028: 0
  - 31.03.2029: 0

- **Redeemable Preference Shares**: 
  - 31.03.2022: 0
  - 31.03.2023: 0
  - 2024-03-31: 0
  - 2025-03-31: 0
  - 2026-03-31: 0
  - 2027-03-31: 0
  - 31.03.2028: 0
  - 31.03.2029: 0

- *

In [10]:
user_question_6 = "What is Debt-to-Equity Ratio?"
result = app.invoke({"question": user_question_6})
print(result["result"])

The Debt-to-Equity Ratio (D/E Ratio) is a financial metric that compares a company's total liabilities to its shareholders' equity. It is used to assess a company's financial leverage and indicates the proportion of debt used to finance the company's assets relative to the equity provided by shareholders.

The formula to calculate the Debt-to-Equity Ratio is:

\[
\text{Debt-to-Equity Ratio} = \frac{\text{Total Liabilities}}{\text{Shareholders' Equity}}
\]

Where:
- **Total Liabilities** includes all of a company's debts and obligations.
- **Shareholders' Equity** is the residual interest in the assets of the company after deducting liabilities, which includes common stock, preferred stock, retained earnings, and additional paid-in capital.

### Interpretation:
- A **higher D/E ratio** indicates that a company is using more debt to finance its operations, which can imply higher risk, especially if the company faces financial difficulties.
- A **lower D/E ratio** suggests that a company 

In [ ]:
user_question_7 = "Calculate the Return on Equity (ROE)? based on provided data"
result = app.invoke({"question": user_question_7})
print(result["result"])

To calculate the Return on Equity (ROE), we need the net income and the average shareholders' equity. However, the provided data does not include net income directly. We can use the net worth as a proxy for shareholders' equity.

ROE is calculated as:
\[ \text{ROE} = \frac{\text{Net Income}}{\text{Average Shareholders' Equity}} \]

Since net income is not provided, we will use the change in net worth as a proxy for net income. The average shareholders' equity can be approximated by averaging the net worth at the beginning and end of the period.

Let's calculate the ROE for the year ending 31.03.2023:

1. **Net Worth at 31.03.2022**: 380.59
2. **Net Worth at 31.03.2023**: 466.53

Change in Net Worth (proxy for net income):
\[ \text{Change in Net Worth} = 466.53 - 380.59 = 85.94 \]

Average Shareholders' Equity:
\[ \text{Average Shareholders' Equity} = \frac{380.59 + 466.53}{2} = 423.56 \]

ROE for the year ending 31.03.2023:
\[ \text{ROE} = \frac{85.94}{423.56} \times 100 = 20.29\% \]

